In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from loader_util.nn.conv import MinVGGNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
import os
sns.set()

In [ ]:
args = {
    'output': '',
    'models': '',
    'num_models': 5
}

In [ ]:
# load trianing and testing data
(trainx,trainy),(testx, testy) = cifar10.load_data()
trainx = trainx.astype('float') / 255.0
testx = testx.astype('float') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

labelNames = ['airplanes', 'automobile', 'bird', 'cat', 'deer',
              'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# initialise image data egenrator
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                         height_shift_range=0.1, horizontal_flip=True,
                         fill_mode='nearest')

In [ ]:
# loop over the no of mdoels to train
total_epochs = 40
for i in range(0, args['num_models']):
    # initialise the opt and model
    print(f"[INFO] training model {i+1}/{args['num_models']}")
    opt = SGD(learning_rate=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
    model = MinVGGNet.build(width=32, height=32, depth=3, classes=10)
    model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])
    
    # train the network
    H = model.fit_generator(aug.flow(trainx, trainy, batch_size=64),
                            validation_data=(testx, testy),
                            epochs=total_epochs,
                            steps_per_epoch=len(trainx) // 64,
                            verbose=1)
    # save model to disk
    p = [args['models'], f"model_{i}.model"]
    model.save(os.path.sep.join(p))
    
    # evaluate the network
    preds = model.predict(testx, batch_size=64)
    report = classification_report(testy.argmax(axis=1),
                                   preds.argmax(axis=1),
                                   target_names=labelNames)
    
    # save the classification report to file
    p = [args['output'], f"model_{i}.txt"]
    f = open(os.path.sep.join(p), "w")
    f.write(report)
    f.close()
    
    
    # now do the accuracy loss plotting
    p = [args['output'], f"model_{i}.png"]
    epochs = range(1,total_epochs+1)
    loss = H.history['loss']
    accuracy = H.history['acc']
    val_loss = H.history['val_loss']
    val_accuracy = H.history['val_acc']
    plot_df = pd.DataFrame(data=np.c_[epochs,loss,accuracy,val_loss,val_accuracy], 
                       columns=['epochs','loss', 'accuracy', 'val_loss', 'val_accuracy'])

    sns.set(font_scale=1)
    f, ax = plt.subplots(1, 1, figsize=(18,10))
    sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
    sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
    ax.set_ylabel('Loss or Accuracy')
    ax.set_xlabel('Epochs')
    plt.setp(ax.get_legend().get_texts(), fontsize='18'); # for legend text
    f.savefig(os.path.sep.join(p))